In [1]:
# Importing Dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

#Import API Key
from api_key import wx_api_key

###  Perform API Calls

In [2]:
# Base URL
base_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"

format_output = 'json'

tp = 6

loc = 'New+york,ny'

# Building Query URL
query_url = f"{base_url}key={wx_api_key}&format={format_output}&tp={tp}&q={loc}&date="

In [3]:
dates = []

number_of_days = 365
start_date_obj = datetime(2015, 1, 1)

for day in range(number_of_days):
    new_date = start_date_obj + timedelta(day)
    new_date = new_date.strftime("%Y-%m-%d")
    dates.append(new_date)

In [4]:
# Creating Empty Lists
date_list = []
time_list = []
tempF = []
DewPointF = []
windspeedMiles = []
WindGustMiles = []
winddir16Point = []
precipInches = []
humidity = []
visibilityMiles = []
pressure = []
cloudcover = []
totalSnow_cm = []

# Using a For Loop to Make Successive API Calls
print('Beginning Data Retrieval')
print('------------------------')

# Loop Through Each Date
for time in range(4):
    # Making Requests        
    for date in dates:
        response = requests.get(query_url + date).json()
    # Using Try/Except to Grab Variables if Available
        try:
            print(f'Processing Time {time} for {date}')
            date_list.append(response['data']['weather'][0]['date'])
            totalSnow_cm.append(response['data']['weather'][0]['totalSnow_cm'])
            time_list.append(response['data']['weather'][0]['hourly'][time]['time'])
            tempF.append(response['data']['weather'][0]['hourly'][time]['tempF'])
            DewPointF.append(response['data']['weather'][0]['hourly'][time]['DewPointF'])
            windspeedMiles.append(response['data']['weather'][0]['hourly'][time]['windspeedMiles'])
            WindGustMiles.append(response['data']['weather'][0]['hourly'][time]['WindGustMiles'])
            winddir16Point.append(response['data']['weather'][0]['hourly'][time]['winddir16Point'])
            precipInches.append(response['data']['weather'][0]['hourly'][time]['precipInches'])
            humidity.append(response['data']['weather'][0]['hourly'][time]['humidity'])
            visibilityMiles.append(response['data']['weather'][0]['hourly'][time]['visibilityMiles'])
            pressure.append(response['data']['weather'][0]['hourly'][time]['pressure'])
            cloudcover.append(response['data']['weather'][0]['hourly'][time]['cloudcover'])
        except KeyError:
            print('Day not found. Skipping...')
print('------------------------')
print('Data Retrieval Complete')
print('------------------------')

Beginning Data Retrieval
------------------------
Processing Time 0 for 2015-01-01
Processing Time 0 for 2015-01-02
Processing Time 0 for 2015-01-03
Processing Time 0 for 2015-01-04
Processing Time 0 for 2015-01-05
Processing Time 0 for 2015-01-06
Processing Time 0 for 2015-01-07
Processing Time 0 for 2015-01-08
Processing Time 0 for 2015-01-09
Processing Time 0 for 2015-01-10
Processing Time 0 for 2015-01-11
Processing Time 0 for 2015-01-12
Processing Time 0 for 2015-01-13
Processing Time 0 for 2015-01-14
Processing Time 0 for 2015-01-15
Processing Time 0 for 2015-01-16
Processing Time 0 for 2015-01-17
Processing Time 0 for 2015-01-18
Processing Time 0 for 2015-01-19
Processing Time 0 for 2015-01-20
Processing Time 0 for 2015-01-21
Processing Time 0 for 2015-01-22
Processing Time 0 for 2015-01-23
Processing Time 0 for 2015-01-24
Processing Time 0 for 2015-01-25
Processing Time 0 for 2015-01-26
Processing Time 0 for 2015-01-27
Processing Time 0 for 2015-01-28
Processing Time 0 for 2015

Processing Time 0 for 2015-09-05
Processing Time 0 for 2015-09-06
Processing Time 0 for 2015-09-07
Processing Time 0 for 2015-09-08
Processing Time 0 for 2015-09-09
Processing Time 0 for 2015-09-10
Processing Time 0 for 2015-09-11
Processing Time 0 for 2015-09-12
Processing Time 0 for 2015-09-13
Processing Time 0 for 2015-09-14
Processing Time 0 for 2015-09-15
Processing Time 0 for 2015-09-16
Processing Time 0 for 2015-09-17
Processing Time 0 for 2015-09-18
Processing Time 0 for 2015-09-19
Processing Time 0 for 2015-09-20
Processing Time 0 for 2015-09-21
Processing Time 0 for 2015-09-22
Processing Time 0 for 2015-09-23
Processing Time 0 for 2015-09-24
Processing Time 0 for 2015-09-25
Processing Time 0 for 2015-09-26
Processing Time 0 for 2015-09-27
Processing Time 0 for 2015-09-28
Processing Time 0 for 2015-09-29
Processing Time 0 for 2015-09-30
Processing Time 0 for 2015-10-01
Processing Time 0 for 2015-10-02
Processing Time 0 for 2015-10-03
Processing Time 0 for 2015-10-04
Processing

Processing Time 1 for 2015-05-12
Processing Time 1 for 2015-05-13
Processing Time 1 for 2015-05-14
Processing Time 1 for 2015-05-15
Processing Time 1 for 2015-05-16
Processing Time 1 for 2015-05-17
Processing Time 1 for 2015-05-18
Processing Time 1 for 2015-05-19
Processing Time 1 for 2015-05-20
Processing Time 1 for 2015-05-21
Processing Time 1 for 2015-05-22
Processing Time 1 for 2015-05-23
Processing Time 1 for 2015-05-24
Processing Time 1 for 2015-05-25
Processing Time 1 for 2015-05-26
Processing Time 1 for 2015-05-27
Processing Time 1 for 2015-05-28
Processing Time 1 for 2015-05-29
Processing Time 1 for 2015-05-30
Processing Time 1 for 2015-05-31
Processing Time 1 for 2015-06-01
Processing Time 1 for 2015-06-02
Processing Time 1 for 2015-06-03
Processing Time 1 for 2015-06-04
Processing Time 1 for 2015-06-05
Processing Time 1 for 2015-06-06
Processing Time 1 for 2015-06-07
Processing Time 1 for 2015-06-08
Processing Time 1 for 2015-06-09
Processing Time 1 for 2015-06-10
Processing

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
 # Creating DataFrame
historical_wx_df = pd.DataFrame({
    'Date': date_list,
    'Time': time_list,
    'Temperature (F)': tempF,
    'Dew Point (F)': DewPointF,
    'Wind Speed (mph)': windspeedMiles,
    'Wind Gusts (mph)': WindGustMiles,
    'Wind Direction': winddir16Point,
    'Precipitation (in)': precipInches,
    'Humidity': humidity,
    'Visibility (mi)': visibilityMiles,
    'Pressure': pressure,
    'Cloud Coverage (%)': cloudcover,
    'Snow Total (cm)': totalSnow_cm
})
historical_wx_df

,Date,Time,Temperature (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity,Visibility (mi),Pressure,Cloud Coverage (%),Snow Total (cm)
0,2015-01-01,0,27,15,10,14,WSW,0.0,59,6,1022,10,0.0
1,2015-01-02,0,34,23,11,20,WSW,0.0,64,6,1017,6,0.0
2,2015-01-03,0,32,25,8,14,NNE,0.0,74,6,1032,100,0.0
3,2015-01-04,0,43,43,3,5,N,0.0,97,1,1018,100,0.0
4,2015-01-05,0,43,38,13,22,W,0.0,83,6,1011,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,2015-10-04,600,53,46,15,20,NE,0.0,75,6,1027,61,0.0
642,2015-10-05,600,51,47,11,14,NNE,0.0,87,6,1023,68,0.0
643,2015-10-06,600,54,45,4,6,NNW,0.0,66,6,1018,13,0.0
644,2015-10-07,600,58,53,5,7,NNW,0.0,80,6,1016,65,0.0


In [ ]:
# requests.get(query_url + dates[0]).json()
# response['data']['weather'][0]['totalSnow_cm']

In [ ]:
# WindGustsMiles
# winddir16Point
# precipInches
# humidity
# visibilityMiles
# pressure
# cloudcover
# totalSnow_cm